# `mbox` Email Analysis

Inspired from [Justin Ellis' "Personal Analytics Part 1: Gmail"](https://jellis18.github.io/post/2018-01-17-mail-analysis/), below is my notebook to analyze my emails.

My specific use-case is I want to be able to quickly go through my inbox and bulk-delete emails. I am particularly interested in finding the 'most frequent senders'.


## Steps
1) Go to [Google Takeout](https://takeout.google.com/settings/takeout?pli=1) and download your `Mail` data only (you can quickly find this by searching the page for `mbox`)
2) Request a download; this can take several hours, so get going on this...
3) Move the `mbox` file to this directory, ensuring the file is named: `All mail Including Spam and Trash.mbox`
4) Run this notebook.

**Current Status:** _In Progress_

In [ ]:
# set raw data mbox path
mboxfile = './Email Analysis/All mail Including Spam and Trash.mbox'

In [ ]:
import csv
import mailbox
import re
import warnings
from typing import List, Dict

import numpy as np
import pandas as pd
from dateutil import parser

## Read in `mbox` file

In [ ]:
mbox = mailbox.mbox(mboxfile)

In [ ]:
# do this as a standalone operation, as it takes >1.5min on a ~6GB mbox file
x = mbox[0]

In [ ]:
for key in x.keys():
    print(key)

## Write out to CSV

In [ ]:
cols: Dict[str, str] = {
    # mbox name: dataframe name
    'Date': 'date',
    'From': 'from',
    'To': 'to',
    'Subject': 'subject',
    'Reply-To': 'reply_to',
    'Delivered-To': 'delivered_to',
    'Message-ID': 'message_id',
    'X-GM-THRID': 'x_gm_thrid',
    'X-Gmail-Labels': 'labels',
    # Misc
    'ARC-Authentication-Results': 'arc-authentication-results',
    'ARC-Message-Signature': 'arc-message-signature',
    'ARC-Seal': 'arc-seal',
    'Authentication-Results': 'authentication-results',
    'Content-Type': 'content-type',
    'DKIM-Signature': 'dkim-signature',
    'Mime-Version': 'mime-version',
    'Received': 'received',
    'Received-SPF': 'received-spf',
    'Return-Path': 'return-path',
    'X-Entity-ID': 'x-entity-id',
    'X-Feedback-ID': 'x-feedback-id',
    'X-Google-Smtp-Source': 'x-google-smtp-source',
    'X-Received': 'x-received',
    'X-SG-EID': 'x-sg-eid',
    'X-SG-ID': 'x-sg-id',
}

## Convert to CSV

In [ ]:
with open("./Email Analysis/mbox.csv", "w") as outfile:
    writer = csv.writer(outfile)
    for message in mbox:
        data = []
        for col in cols.keys():
            data.append(message[col])
        writer.writerow(data)

In [ ]:
df = pd.read_csv('./Email Analysis/mbox.csv', names=cols.values())

## Data Cleaning

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=parser.UnknownTimezoneWarning)
    df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True)

df

In [ ]:
def clean_emails(string):
    if string is np.NaN:
        return ""
    email = re.findall(r'<(.+?)>', string)
    if not email:
        email = list(filter(lambda y: '@' in y, string.split()))
    return email[0] if email else string


for col in [
    'from',
    'to',
    'reply_to',
    'delivered_to',
]:
    df.loc[:, col] = df.loc[:, col].apply(lambda x: clean_emails(x))

domain_only = lambda x: x.split("@")[1] if '@' in x else x
df.loc[:, 'from_domain'] = df.loc[:, 'from'].apply(domain_only)
df.loc[:, 'to_domain'] = df.loc[:, 'to'].apply(domain_only)

tld_domain_only = lambda x: ".".join(x.split(".")[-2:]) if '.' in x else x
df.loc[:, 'from_tld_domain'] = df.loc[:, 'from_domain'].apply(tld_domain_only)
df.loc[:, 'to_tld_domain'] = df.loc[:, 'to_domain'].apply(tld_domain_only)


def clean_labels(row) -> List[str]:
    if isinstance(row, list):
        return row
    if row is np.NAN:
        return []
    if isinstance(row, str) and ',' in row:
        return [label.strip() for label in row.split(',')]
    return [row]


df.loc[:, 'labels'] = df.loc[:, 'labels'].apply(lambda row: clean_labels(row))

df = df[df['date'].notna()]

## Process

In [ ]:
df.head()

In [ ]:
# date_filter = (df['date'].dt.tz_localize(None) >= (pd.to_datetime('today').normalize() - pd.DateOffset(months=60)))
date_filter = True

inbox_not_trash = (df['labels'].apply(lambda x: 'Trash' not in x and 'Inbox' in x))

In [ ]:
for col in ['from','from_domain','from_tld_domain']:
    display(pd.DataFrame(df[date_filter & inbox_not_trash].groupby([col]).size().sort_values(ascending=False), columns=['count']))